In [1]:
!curl https://zenodo.org/record/3974431/files/vanilla_bert_tiny_on_MSMARCO.tar.gz >> vanilla_bert_tiny_on_MSMARCO.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.2M  100 42.2M    0     0  7181k      0  0:00:06  0:00:06 --:--:-- 9226k


In [2]:
!tar -zxvf vanilla_bert_tiny_on_MSMARCO.tar.gz

bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/
bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000.data-00000-of-00001
bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000.meta
bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000.index


In [27]:
!curl https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-2_H-128_A-2.zip >> bert.zip
!unzip bert.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.7M  100 15.7M    0     0  6607k      0  0:00:02  0:00:02 --:--:-- 6610k
Archive:  bert.zip
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_config.json        
replace vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [53]:
import pyterrier as pt
if not pt.started():
    pt.init(version="snapshot")
    
import tensorflow as tf

from pyterrier.transformer import TransformerBase

from run_reranking import model_fn_builder
from input_parser_pyterrier import input_fn_builder
from bert.modeling import BertConfig

In [54]:
class PARADEPipeline(TransformerBase):
    def __init__(self, aggregation_method):
        self.aggregation_method = aggregation_method #'cls_max',  'cls_avg', 'cls_attn' or 'cls_transformer'
        
        self.run_config = tf.estimator.tpu.RunConfig(
            cluster=None,
            model_dir=None,
            save_checkpoints_steps=1000,
            keep_checkpoint_max=1,
            tpu_config=None)
            
        '''
        tpu_config is set to None since we aren't using a tpu, but in case we actually need this
        i'll just keep this commented out instead of deleting it
            
            tpu_config=tf.estimator.tpu.TPUConfig(
                iterations_per_loop=1000,
                num_shards=8,
                per_host_input_for_training=is_per_host))
        '''
        
        self.model_fn = model_fn_builder(
            bert_config=BertConfig.from_json_file('bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/bert_config.json'),
            num_labels=2,
            init_checkpoint='bert_models_onMSMARCO/vanilla_bert_tiny_on_MSMARCO/model.ckpt-1600000',
            learning_rate=5e-5,
            num_train_steps=None,
            num_warmup_steps=None,
            use_tpu=False,
            use_one_hot_embeddings=False,
            aggregation_method=self.aggregation_method,
            pretrained_model='bert',
            from_distilled_student=False)
        
        self.estimator = tf.estimator.tpu.TPUEstimator(
            use_tpu=False,
            model_fn=self.model_fn,
            config=self.run_config,
            train_batch_size=32,
            eval_batch_size=32,
            predict_batch_size=32)
        
        
    def transform(self, queries_and_docs):
        eval_input_fn = input_fn_builder(
            dataset_df=queries_and_docs,
            max_num_segments_perdoc=8,
            max_seq_length=128,
            is_training=False)
            
        result = self.estimator.predict(input_fn=eval_input_fn, yield_single_examples=True)
        
        results = []
        for item in result:
            print(item)
            

In [55]:
import pandas as pd
q = "chemical reactions"
doc1 = "professor proton demonstrated the chemical reaction"
doc2 = "chemical brothers is great techno music"

df = pd.DataFrame([["q1", q, "doc1", doc1], ["q1", q, "doc2", doc2]], columns=["qid", "query", "docno", "body"])
df

,qid,query,docno,body
0,q1,chemical reactions,doc1,professor proton demonstrated the chemical rea...
1,q1,chemical reactions,doc2,chemical brothers is great techno music


In [56]:
pipeline = PARADEPipeline(aggregation_method='cls_max')

#vaswani  = pt.datasets.get_dataset("vaswani")
#vaswani.get_corpus()


pipeline(df)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2pa9va6b', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f28c9276c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None,

ValueError: in converted code:

    /tf/PARADE/input_parser.py:32 extract_fn  *
        sample = tf.parse_single_example(data_record, features)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/parsing_ops.py:1019 parse_single_example
        serialized, features, example_names, name
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/parsing_ops.py:1063 parse_single_example_v2_unoptimized
        return parse_single_example_v2(serialized, features, name)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/parsing_ops.py:2093 parse_single_example_v2
        dense_defaults, dense_shapes, name)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/parsing_ops.py:2210 _parse_single_example_v2_raw
        name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/gen_parsing_ops.py:1201 parse_single_example
        dense_shapes=dense_shapes, name=name)
    /usr/local/lib/python3.6/dist-packages/te